In [15]:
from pyspark.sql.functions import size

In [21]:
from pyspark.sql.functions import array_contains

In [1]:
reader = sqlContext.read.format('com.databricks.spark.avro')

In [2]:
df = reader.load('data/spark_metadata.avro')

In [3]:
df.columns

['id',
 'path',
 'message_id',
 'date',
 'from',
 'to',
 'cc',
 'bcc',
 'subject',
 'references']

In [22]:
selection1 = df[df['from'].startswith('maggie')]
#selection2 = selection1[selection1['to'].apply(lambda name: name.startswith('kev'))]
#selection2.select(['id', 'from', 'to', 'subject']).show()
# col = selection1['to']
selection1.show()

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    id|                path|          message_id|                date|                from|                  to|                  cc|                 bcc|             subject|references|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|486383|maildir/lokay-m/a...|<10497489.1075844...|2001-01-08T08:34:...|maggie.matheson@e...|[kevin.hyatt@enro...|                  []|                  []|Re: TW Security A...|        []|
|490755|maildir/lokay-m/s...|<18395121.1075844...|2001-01-08T08:34:...|maggie.matheson@e...|[kevin.hyatt@enro...|                  []|                  []|Re: TW Security A...|        []|
|505280|maildir/mcconnell...|<14851779.1075861...|2001-11-15

In [29]:
from pyspark.sql.functions import udf

In [30]:
from pyspark.sql.types import BooleanType

In [40]:
# selection1[col.getItem(0).startswith('kevin')].show()
# selection1.select(size(col)).show()

kevudf = udf(hasKevin, BooleanType())
selection1.filter(kevudf(df.to)).show()

+------+--------------------+--------------------+--------------------+--------------------+--------------------+---+---+--------------------+----------+
|    id|                path|          message_id|                date|                from|                  to| cc|bcc|             subject|references|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+---+---+--------------------+----------+
|486383|maildir/lokay-m/a...|<10497489.1075844...|2001-01-08T08:34:...|maggie.matheson@e...|[kevin.hyatt@enro...| []| []|Re: TW Security A...|        []|
|490755|maildir/lokay-m/s...|<18395121.1075844...|2001-01-08T08:34:...|maggie.matheson@e...|[kevin.hyatt@enro...| []| []|Re: TW Security A...|        []|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+---+---+--------------------+----------+



In [20]:
df1 = df.toPandas()

In [57]:
def array_any(f, col):
    return any(map(f, col))

def lambda_startswith(prefix):
    return lambda value: value.startswith(prefix)

def hasKevin(tos):
    return array_any(lambda_startswith('kev'), tos)

def contains(value):
    return udf(lambda name: value in name, BooleanType())

df2 = df1[df1['from'].apply(lambda_startswith('maggie'))]

selection2 = df2[df2['to'].apply(hasKevin)]
selection2[['from', 'to', 'subject']]


,from,to,subject
21856,maggie.matheson@enron.com,"[kevin.hyatt@enron.com, michelle.lokay@enron.com]",Re: TW Security Access Request
26228,maggie.matheson@enron.com,"[kevin.hyatt@enron.com, michelle.lokay@enron.com]",Re: TW Security Access Request


In [ ]:
selection2['to']

In [36]:
df1[df1['from'].apply(lambda x: x.startswith('wally'))]

,id,path,message_id,date,from,to,cc,bcc,subject,references
456234,33523,maildir/lay-k/deleted_items/472.,<614215.1075860820728.JavaMail.evans@thyme>,2002-01-30T09:42:14-08:00,wallymar@home.com,[klay@enron.com],[],[],Demand Ken Lay Donate Proceeds from Enron Stoc...,[]
492401,69690,maildir/taylor-m/archive/2001_11/49.,<26135252.1075861971244.JavaMail.evans@thyme>,2001-11-07T15:06:58-08:00,wally.shaw@blakes.com,[legal <.taylor@enron.com>],[],[],Fwd: Re: Acknowledgments and Resolutions,[]
492403,69692,maildir/taylor-m/archive/2001_11/51.,<22238007.1075861971290.JavaMail.evans@thyme>,2001-11-07T14:30:16-08:00,wally.shaw@blakes.com,"[douglas.stephen@enron.com, legal <.taylor@enr...",[],[],Acknowledgments and Resolutions,[]


In [63]:
df.select(df['from'])[~contains('enron')(df['from'])].distinct().count()

13782

# Load CSV

In [9]:
dicFile = 'enron_small_dic.csv'

csvLoader = sqlContext.read.format('com.databricks.spark.csv')
dic = csvLoader.options(delimiter='\t', header='false', inferschema='true').load(dicFile)

In [13]:
dic.columns

['C0', 'C1', 'C2']